In [1]:
import numpy as np
import plotly.express as px

## Аналитическое решение и производные

Для начала упростим функцию

$f(x) = \frac{1}{2} [x_1^2 + x_3^2 + (x_1-x_2)^2 + (x_2-x_3)^2] - x_1$

$f(x) = \frac{1}{2} [x_1^2 + x_3^2 + x_1^2 - 2x_1x_2 + x_2^2 + x_2^2 - 2x_2x_3 + x_3^2] - x_1$

$f(x) = \frac{1}{2} [2x_1^2 + 2x_2^2 + 2x_3^2 - 2x_1x_2 - 2x_2x_3] - x_1$

$f(x) = x_1^2 + x_2^2 + x_3^2 - x_1x_2 - x_2x_3 - x_1$

Найдем производные

$grad = \begin{bmatrix} 2x_1-x_2-1 \\ 2x_2-x_1-x_3 \\ 2x_3-x_2 \end{bmatrix}$

$H = \begin{bmatrix} 2&-1&0 \\ -1&2&-1 \\ 0&-1&2 \end{bmatrix}$

Матрица H положительно определена, значит функция вогнутая (у нас говорили "выпуклая вверх" и "выпуклая вниз"), поэтому применять метод можно

Аналитическое решение оптимизации

$x = \begin{bmatrix} 0.75 \\ 0.5 \\ 0.25\end{bmatrix}; f(x) = -0.375$

In [2]:
def task_function(x):
    return np.square(x).sum() - x[0]*x[1] - x[1]*x[2] - x[0]

def task_true_grad(x):
    return np.array([
        2*x[0] - x[1] - 1,
        2*x[1] - x[0] - x[2],
        2*x[2] - x[1]
    ])

# Так как функции внутри градиента линейные,
# можно также было использовать np.dot, 
# но это частный случай + читается хуже

def task_true_H(x):
    return np.array([
        [ 2, -1,  0],
        [-1,  2, -1],
        [ 0, -1,  2]
    ])

## Вспомогательные функции

In [3]:
def dot3(a, b, c):
    return np.dot(np.dot(a, b), c)

def e(k, i):
    """
    Возвращает массив из k элементов,
    на индексе i стоит единица,
    все остальные - нули
    """
    return np.eye(k)[i]

## Вспомогательные классы

In [4]:
# Все, связанное с целевой функцией,
# выделим в отдельный класс

class Function:
    
    def __init__(self, 
                 f, 
                 true_grad, 
                 true_H, 
                 use_numerical: dict):
        self._f = f
        # true_ - аналитический
        self.true_grad = true_grad
        self.true_H = true_H
        
        # Использовать ли численный метод вычисления производных
        self.use_numerical = use_numerical
        # Число вызовов
        # (функции, градиента, гессиана)
        self.n_calls = {
            'f': 0, 'grad': 0, 'H': 0
        }
        
    def __call__(self, x):
        self.n_calls['f'] += 1
        return self._f(x)
    
    def f(self, x):
        self.n_calls['f'] += 1
        return self._f(x)

    def grad(self, x, epsilon=1e-4):
        self.n_calls['grad'] += 1

        if self.use_numerical['grad']:
            return self.diff_grad(x, epsilon)
        else:
            return self.true_grad(x)
    

    def H(self, x, epsilon=1e-4):
        self.n_calls['H'] += 1
        
        if self.use_numerical['H']:
            return self.diff_H(x, epsilon)
        else:
            return self.true_H(x)


    def diff_grad(self, x: np.array, h: float):
        """
        Градиент методом разностей
        """
        k = x.shape[0]
        return np.array([
            # пишем self(x), потому что есть __call__
            self(x + e(k, i)*h) - \
            self(x - e(k, i)*h)
            for i in range(k)
        ]) / (2*h)


    def diff_H(self, x: np.array, h: float):
        """
        Вторые производные методом разностей
        """
        k = x.shape[0]
        return np.array([[
            self(x + e(k, i)*h + e(k, j)*h) - \
            self(x + e(k, i)*h - e(k, j)*h) - \
            self(x - e(k, i)*h + e(k, j)*h) + \
            self(x - e(k, i)*h - e(k, j)*h)
            for i in range(k)
        ]
            for j in range(k)
        ]) / (4*h**2)

In [5]:
# Также выделим в отдельный класс все,
# что связано с поиском параметра alpha

class AlphaFinder:
    
    def __init__(self):
        pass
    
    @staticmethod
    def Wolfe_condition_1(f, x, alpha, p, c1):
        lhs = f(x + alpha*p)
        rhs = f(x) + c1*alpha*np.dot(f.grad(x), p)
        return lhs < rhs
    
    @staticmethod
    def Wolfe_condition_2(f, x, alpha, p, c2):
        lhs = np.dot(f.grad(x + alpha*p), p)
        rhs = c2 * alpha * np.dot(f.grad(x), p)
        return lhs >= rhs
    
    def __call__(self, 
                 f, x, p, 
                 c1=0.01, c2=0.90, 
                 alpha_start=0.1, alpha_step=0.1):
        """
        Линейный поиск параметра alpha
        """
        alpha = alpha_start
        while not (
            self.Wolfe_condition_1(f, x, alpha, p, c1) and 
            self.Wolfe_condition_2(f, x, alpha, p, c2)
        ):
            alpha += alpha_step
        return alpha
        

## Метод BFGS (без приближения H)

In [6]:
def BFGS_method(f: Function, 
                x0: np.array,
                epsilon: float):
    
    x = x0.copy()
    alpha_finder = AlphaFinder()
    # Для записи статистики
    x_history = [x0]
    y_history = [f(x0)]

    while True:
        # Основное
        grad = f.grad(x)
        H = f.H(x)
        H_inv = np.linalg.inv(H)
        # Направление движения
        p = -1 * np.dot(H_inv, grad)
        # Поиск длины шага alpha
        alpha = alpha_finder(f, x, p, 
                             alpha_start=0.1, alpha_step=0.1)
        # Делаем шаг
        x = x + alpha * p
        y = f(x)
        # Обновляем статистику
        x_history.append(x.copy())
        y_history.append(y.copy())
        
        # Проверяем критерий остановки
        if np.linalg.norm(grad) < epsilon:
            return x, y, x_history, y_history
    

## Метод BFGS (приближение H)

In [7]:
def update_H(H, s, y):
    """
    Обновление матрицы Гессе
    """
    k = H.shape[0]
    I = np.eye(k)

    ro = 1 / np.dot(y, s)
    parente1 = (I - ro * np.outer(s, y))
    parente2 = (I - ro * np.outer(y, s))
    add = ro * np.outer(s, s)
    
    return dot3(parente1, H, parente2) + add

In [8]:
def BFGS_method_H_evolve(f: Function, 
                         x0: np.array,
                         epsilon: float):
    
    x = x0.copy()
    alpha_finder = AlphaFinder()
    
    # Для записи статистики
    x_history = [x0]
    y_history = [f(x0)]

    k = x0.shape[0]
    H = np.eye(k)
    
    while True:
        # Основное
        grad = f.grad(x)
        H = f.H(x)
        H_inv = np.linalg.inv(H)
        # Направление движения
        p = -1 * np.dot(H_inv, grad)
        # Поиск длины шага alpha
        alpha = alpha_finder(f, x, p, 
                             alpha_start=0.1, alpha_step=0.1)
        step = alpha * p

        # Обновляем H
        s = step
        y = f.grad(x+step) - f.grad(x)
        H = update_H(H, s, y)

        # Делаем шаг
        x = x + alpha * p
        y = f(x)
        # Обновляем статистику
        x_history.append(x.copy())
        y_history.append(y.copy())
        
        # Проверяем критерий остановки
        if np.linalg.norm(grad) < epsilon:
            return x, y, x_history, y_history
    

## Тест метода

### Аналитические производные

In [9]:
x0 = np.array([200, 500, 300])
epsilon = 1e-3

In [10]:
func_true = Function(
    f=task_function,
    true_grad=task_true_grad,
    true_H=task_true_H,
    use_numerical={'grad': False, 'H': False}
)

In [11]:
%%time
x, y, x_history, y_history = BFGS_method(func_true, x0, epsilon)

CPU times: total: 15.6 ms
Wall time: 34.7 ms


In [12]:
# Найденные значения
y, x

(-0.37499997605605695, array([0.75008558, 0.50021453, 0.25012874]))

In [13]:
# Число вызовов
func_true.n_calls

{'f': 209, 'grad': 304, 'H': 16}

In [ ]:
# Эволюция параметров по итерациям
a, b, c = np.vstack(x_history).T
fig = px.line(y=[a, b, c], 
              template='plotly_dark', 
              log_y=True)
fig.update_layout(
    margin_t=0,margin_b=0,
    margin_l=0,margin_r=0
)
fig.show()

In [ ]:
# Изменение целевой функции по итерациям
# Оптимизация заняла всего 10 шагов
px.line(y_history, template='plotly_dark')

## Численный только Гессиан

In [14]:
func_H_num = Function(
    f=task_function,
    true_grad=task_true_grad,
    true_H=task_true_H,
    use_numerical={'grad': False, 'H': True}
)

In [15]:
%%time
x, y, x_history, y_history = BFGS_method(func_H_num, x0, epsilon)

CPU times: total: 31.2 ms
Wall time: 24.6 ms


In [16]:
# Число вызовов
func_H_num.n_calls

{'f': 785, 'grad': 304, 'H': 16}

### Численные производные

In [17]:
# x0 и epsilon такие же

In [18]:
func_num = Function(
    f=task_function,
    true_grad=task_true_grad,
    true_H=task_true_H,
    use_numerical={'grad': True, 'H': True}
)

In [19]:
%%time
x, y, x_history, y_history = BFGS_method(func_num, x0, epsilon)

CPU times: total: 78.1 ms
Wall time: 79.9 ms


In [20]:
y, x

(-0.3749999762488081, array([0.75008504, 0.50021366, 0.25012805]))

In [21]:
# Число вызовов
func_num.n_calls

{'f': 2609, 'grad': 304, 'H': 16}

In [ ]:
# Эволюция параметров по итерациям
a, b, c = np.vstack(x_history).T
fig = px.line(y=[a, b, c], 
              template='plotly_dark', 
              log_y=True)
fig.update_layout(
    margin_t=0,margin_b=0,
    margin_l=0,margin_r=0
)
fig.show()

In [ ]:
# Изменение целевой функции по итерациям
px.line(y_history, template='plotly_dark')

### C эволюцией H

In [22]:
# x0 и epsilon такие же

In [23]:
func_H = Function(
    f=task_function,
    true_grad=task_true_grad,
    true_H=task_true_H,
    use_numerical={'grad': False, 'H': True}
)

In [24]:
%%time
x, y, x_history, y_history = BFGS_method_H_evolve(func_num, x0, epsilon)

CPU times: total: 78.1 ms
Wall time: 79.2 ms


In [25]:
y, x

(-0.3749999762488081, array([0.75008504, 0.50021366, 0.25012805]))

In [26]:
# Число вызовов
func_num.n_calls

{'f': 5410, 'grad': 640, 'H': 32}

In [ ]:
# Эволюция параметров по итерациям
a, b, c = np.vstack(x_history).T
fig = px.line(y=[a, b, c], 
              template='plotly_dark', 
              log_y=True)
fig.update_layout(
    margin_t=0,margin_b=0,
    margin_l=0,margin_r=0
)
fig.show()

In [ ]:
# Изменение целевой функции по итерациям
px.line(y_history, template='plotly_dark')

Все реализации находят решение с нужной точностью за 16 итераций.

Численные методы тратят заметно больше вызовов фунукций.

При этом численные методы находят почти точные производные:

In [27]:
func_num.grad(x0), func_true.grad(x0)

(array([-100.99999985,  499.99999959,   99.9999998 ]),
 array([-101,  500,  100]))